<a href="https://colab.research.google.com/github/SeymaNurAksoy/CurrencyConverterApp/blob/main/deepLearning0305.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM
from tensorflow.keras.optimizers import Adam 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import load_model
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [ ]:
dataset = pd.read_excel('sentimentAnalysis.xlsx', sheet_name = 'Sheet1')
dataset

,Text,Sentiment
0,Daha öncede almıştım bu cihazdan ense ve sakal...,1
1,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
2,Daha öncede aynısını almıştım çok güzel ve kal...,1
3,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
4,ürün gerçekten çok güzel,1
...,...,...
34187,J7 Pro Cihazı Geldi Fakat Faturası Gelmedi!,0
34188,Müşteri Hizmetlerine Ulaşamama Sorunu!,0
34189,Para İadesi Sorunu!,0
34190,Mağdur Ediyor!,0


In [ ]:
dataset['Text'] = dataset['Text'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))
dataset['Text'] = dataset['Text'].apply(lambda x: x.lower())
dataset['Text'] = dataset['Text'].apply(lambda x: x.strip())

def token(values):
    words = nltk.tokenize.word_tokenize(values)
    filtered_words = [word for word in words if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

dataset['Text'] = dataset['Text'].apply(lambda x: token(x))

In [ ]:
dataset['Text']

0        öncede almıştım cihazdan ense sakal tüketmek o...
1        ürün gayet başarılı sakal kesmede başlık sayıs...
2         öncede aynısını almıştım güzel kaliteli bir ürün
3        erkek kuaförüyüm ense sıfır sakal traşı uygun ...
4                                     ürün gerçekten güzel
                               ...                        
34187           j7 pro cihazı geldi fakat faturası gelmedi
34188                müşteri hizmetlerine ulaşamama sorunu
34189                                  para i̇adesi sorunu
34190                                        mağdur ediyor
34191               ürünü aldığı halde parayı i̇ade etmedi
Name: Text, Length: 34192, dtype: object

In [ ]:
data = dataset['Text'].values.tolist()
sentiment = dataset['Sentiment'].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,sentiment,test_size = 0.2, random_state = 42)

In [ ]:
x_train[290]

'ürün güzel tabanı ince bir saat bir yürüme mesafesinde ayaklarınızı ağrıtıyor'

In [ ]:
y_train[290]

0

In [ ]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(data)
tokenizer.word_index

{'bir': 1,
 'ürün': 2,
 'güzel': 3,
 'iyi': 4,
 'tavsiye': 5,
 'gayet': 6,
 'ederim': 7,
 'aldım': 8,
 'yok': 9,
 'geldi': 10,
 'hızlı': 11,
 'değil': 12,
 'göre': 13,
 'ürünü': 14,
 'olarak': 15,
 'var': 16,
 'kaliteli': 17,
 'kadar': 18,
 '0': 19,
 'telefon': 20,
 'uygun': 21,
 'teşekkürler': 22,
 'cok': 23,
 'biraz': 24,
 'ben': 25,
 'kargo': 26,
 'kullanışlı': 27,
 'fiyat': 28,
 'elime': 29,
 '1': 30,
 'küçük': 31,
 'fiyata': 32,
 'bi': 33,
 'gün': 34,
 'kalitesi': 35,
 'kesinlikle': 36,
 'fiyatına': 37,
 'şarj': 38,
 'sonra': 39,
 'gerçekten': 40,
 '3': 41,
 'bile': 42,
 'fakat': 43,
 'tam': 44,
 'memnun': 45,
 'tek': 46,
 'kötü': 47,
 'kullanıyorum': 48,
 'oldu': 49,
 'bence': 50,
 'iade': 51,
 'performans': 52,
 'fazla': 53,
 'ses': 54,
 'olması': 55,
 'ulaştı': 56,
 'ancak': 57,
 'uzun': 58,
 'harika': 59,
 'önce': 60,
 'sorun': 61,
 'sipariş': 62,
 'mükemmel': 63,
 'büyük': 64,
 'fiyatı': 65,
 'ürünün': 66,
 'başarılı': 67,
 'oldukça': 68,
 'aynı': 69,
 'urun': 70,
 'ayrıca': 

In [ ]:

x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [ ]:
x_train_tokens[290]

[2, 3, 856, 217, 1, 92, 1, 4088]

In [ ]:
x_train[290]

'ürün güzel tabanı ince bir saat bir yürüme mesafesinde ayaklarınızı ağrıtıyor'

In [ ]:

num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

13.149303930744034

In [ ]:
np.max(num_tokens)

229

In [ ]:

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

42

In [ ]:

np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9608095460926532

In [ ]:

x_train_pad = np.array(pad_sequences(x_train_tokens, maxlen=max_tokens))
x_test_pad = np.array(pad_sequences(x_test_tokens, maxlen=max_tokens))


In [ ]:
np.array(x_train_pad[3027])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [ ]:
x_test_pad[2000]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, 36,  4], dtype=int32)

In [ ]:

idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))


def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
model =Sequential()

embedding_size = 50


model.add(Embedding(input_dim=10000,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [ ]:
!pip install tensorflow==2.2.0rc2
!pip install keras==2.2.4

     |████████████████████████████████| 516.2 MB 2.1 kB/s 
     |████████████████████████████████| 3.0 MB 37.7 MB/s 
     |████████████████████████████████| 454 kB 40.0 MB/s 
     |████████████████████████████████| 2.9 MB 25.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

     |████████████████████████████████| 312 kB 5.0 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [ ]:

model.add(LSTM(units=16, return_sequences=True))

model.add(LSTM(units=8, return_sequences=True))

model.add(LSTM(units=4))

model.add(Dense(1, activation='sigmoid'))

In [ ]:

adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 42, 50)            500000    
_________________________________________________________________
lstm (LSTM)                  (None, 42, 16)            4288      
_________________________________________________________________
lstm_1 (LSTM)                (None, 42, 8)             800       
_________________________________________________________________
lstm_2 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 505,301
Trainable params: 505,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:

x_train_pad=np.array(x_train_pad)
y_train =np.array(y_train)

history = model.fit(x_train_pad, y_train, epochs=100, batch_size=64,validation_split=0.25)

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
321/321 [==============================] - ETA: 0s - loss: 0.4578 - accuracy: 0.8034WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fc4a1888ef0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set t

In [ ]:
model.save("sentiment_model.h5")

In [ ]:

x_test_pad=np.array(x_test_pad)
y_test =np.array(y_test)
result = model.evaluate(x_test_pad, y_test)

In [ ]:
result[1]

In [ ]:
y_pred = model.predict_classes(x=x_test_pad[:700])


In [ ]:
y_pred = y_pred.T[0]

In [ ]:
y_pred

In [ ]:
text = "güzel"
texts = [text]
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens,maxlen = max_tokens)
tokens_pad.shape

In [ ]:
print("moralinizin {} olduğunu görüyorum. ".format(model.predict_classes(tokens_pad).tolist()[0][0]))


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.save("sentiment_model.h5")